# DTSC670: Foundations of Machine Learning Models

## Assignment 2: COVID-19 Data Wrangling

#### Name:


### CodeGrade
Please note that this assignment will be manually graded.  **You will have one attempt at submitting this assignment.**

Since we are merely using CodeGrade for this assignment as a platform to manually grade assignments, CodeGrade will not  execute your code for this assignment. Rather, it simply renders your Jupyter Notebook in the web browser. Therefore, any code blocks that generate output must be fully executed within the notebook prior to submission. Otherwise, we will not see the output of your code!

### Details

The purpose of this assignment is to hone your data wrangling skills. Your task for this assignment is to perform the data preparation as instructed in the `DTSC670_Assignment_2` pdf listed in Brightspace. After performing all the data preparation tasks outlined in the document, run the code in the "Prepare DataFrames for Grading" section.  

You are supplied an Excel file called `BrazilCOVIDData.xlsx` - be sure to put the data file in the same directory as this Jupyter Notebook. *Please note that it may take around 5 minutes to read-in all of the data in this file.*


In [1]:
### ENTER CODE HERE ###
import pandas as pd
import numpy as np

# Insert as many cells as you need, but be sure your code is very neat and very well documented.

In [2]:
#load each tab of the Excel file into a DataFrame
fileName = 'BrazilCOVIDData.xlsx'
xls = pd.ExcelFile(fileName)
COVID_data = pd.read_excel(xls, 'Brazil Covid-19 data')
temp_by_state = pd.read_excel(xls, 'Temperature by State')
area = pd.read_excel(xls, 'City area')

In [3]:
#remove extra rows around column names in Excel sheet and adjust some names for simplicity
state_stats = pd.read_excel(xls, 'Brazil State Stats', header=4).drop([0,1],axis=0).reset_index(drop=True).rename({"Common Two Letter":'State','State':'State_Name'},axis=1)

#drop extraneous columns from COVID data
COVID_data = COVID_data.drop(['State-code','Municipality-code','Health-region-code','Health-region-name','Week #','New cases',
                             'Accumulated deaths','New deaths','New Recoveries','New followups (?)','Interior/Metropolitan'],axis=1)

In [4]:
#add the city area to the state stats df and drop extraneous columns
state_stats = state_stats.merge(area, left_on='State', right_on='ST').drop(['ST','Region','Size','Population','% Pop.','Number of Municipal Districts','Per Capita GNP in Reais (R$)','Life'],axis=1) 
#I originally dropped the extra City column as well, but noticed one capital was spelled different so it was easier to use the new column 
state_stats

,State,State_Name,Capitol City,City,SQ_KM
0,AC,Acre,Rio Branco,Rio Branco,8835.000
1,AL,Alagoas,Maceió,Maceió,511.000
2,AP,Amapá,Macapá,Macapá,6407.000
3,AM,Amazonas,Manaus,Manaus,11400.000
4,BA,Bahia,Salvador,Salvador,693.800
5,CE,Ceará,Fortaleza,Fortaleza,313.800
6,DF,Distrito Federal,Brasília,Brasília,5802.000
7,ES,Espírito Santo,Vitória,Vitória,93.380
8,GO,Goiás,Goiânia,Goiânia,739.000
9,MA,Maranhão,São Luís,São Luís,827.141


In [18]:
#get a list of just states and capitals
state_capital_tuples = list(state_stats[['State','City']].itertuples(index=False,name=None))
state_capital_tuples

[('AC', 'Rio Branco'),
 ('AL', 'Maceió'),
 ('AP', 'Macapá'),
 ('AM', 'Manaus'),
 ('BA', 'Salvador'),
 ('CE', 'Fortaleza'),
 ('DF', 'Brasília'),
 ('ES', 'Vitória'),
 ('GO', 'Goiânia'),
 ('MA', 'São Luís'),
 ('MT', 'Cuiabá'),
 ('MS', 'Campo Grande'),
 ('MG', 'Belo Horizonte'),
 ('PA', 'Belém'),
 ('PB', 'João Pessoa'),
 ('PR', 'Curitiba'),
 ('PE', 'Recife'),
 ('PI', 'Teresina'),
 ('RJ', 'Rio de Janeiro'),
 ('RN', 'Natal'),
 ('RS', 'Porto Alegre'),
 ('RO', 'Porto Velho'),
 ('RR', 'Boa Vista'),
 ('SC', 'Florianópolis'),
 ('SP', 'São Paulo'),
 ('SE', 'Aracaju'),
 ('TO', 'Palmas')]

In [19]:
#create a DataFrame from the tuples
state_capital_df = pd.DataFrame(state_capital_tuples, columns = ['State','Municipality'])
state_capital_df

,State,Municipality
0,AC,Rio Branco
1,AL,Maceió
2,AP,Macapá
3,AM,Manaus
4,BA,Salvador
5,CE,Fortaleza
6,DF,Brasília
7,ES,Vitória
8,GO,Goiânia
9,MA,São Luís


In [6]:
#select only the rows for correct capital cities by merging with the df made of state/capital tuples
#some cities appear in other states as well, but are not the capital
#hence needing to select it via tuples of the state/capital combo
COVID_data_caps = COVID_data.merge(state_capital_df)

#merge state stats with the new smaller COVID data
COVID_data_caps = COVID_data_caps.merge(state_stats, on=['State'])
COVID_data_caps

,Region,State,Municipality,Date,Population as of 2019,Accumulated cases,State_Name,Capitol City,City,SQ_KM
0,Norte,RO,Porto Velho,2020-03-27,529544,0,Rondônia,Porto Velho,Porto Velho,34091.0
1,Norte,RO,Porto Velho,2020-03-28,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0
2,Norte,RO,Porto Velho,2020-03-29,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0
3,Norte,RO,Porto Velho,2020-03-30,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0
4,Norte,RO,Porto Velho,2020-03-31,529544,6,Rondônia,Porto Velho,Porto Velho,34091.0
...,...,...,...,...,...,...,...,...,...,...
4072,Centro-Oeste,DF,Brasília,2020-08-20,3015268,143759,Distrito Federal,Brasília,Brasília,5802.0
4073,Centro-Oeste,DF,Brasília,2020-08-21,3015268,145452,Distrito Federal,Brasília,Brasília,5802.0
4074,Centro-Oeste,DF,Brasília,2020-08-22,3015268,147127,Distrito Federal,Brasília,Brasília,5802.0
4075,Centro-Oeste,DF,Brasília,2020-08-23,3015268,148998,Distrito Federal,Brasília,Brasília,5802.0


In [7]:
#create a list counting from 0-150 for each state
days = []
for state in state_stats['State'].tolist():
    for x in range(151):
        days.append(x)


In [8]:
#sort by state and date so days list can be added (I think it was already sorted but I don'trust it)
COVID_data_caps = COVID_data_caps.sort_values(by=['State','Date'])
COVID_data_caps['days']=days

#days_sq = days*days
COVID_data_caps['days_sq']=COVID_data_caps['days']*COVID_data_caps['days']

#days_cube = days*days*days = days_sq*days
COVID_data_caps['days_cube']=COVID_data_caps['days_sq']*COVID_data_caps['days']


#resort back to original order
COVID_data_caps = COVID_data_caps.sort_index()

COVID_data_caps

,Region,State,Municipality,Date,Population as of 2019,Accumulated cases,State_Name,Capitol City,City,SQ_KM,days,days_sq,days_cube
0,Norte,RO,Porto Velho,2020-03-27,529544,0,Rondônia,Porto Velho,Porto Velho,34091.0,0,0,0
1,Norte,RO,Porto Velho,2020-03-28,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,1,1,1
2,Norte,RO,Porto Velho,2020-03-29,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,2,4,8
3,Norte,RO,Porto Velho,2020-03-30,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,3,9,27
4,Norte,RO,Porto Velho,2020-03-31,529544,6,Rondônia,Porto Velho,Porto Velho,34091.0,4,16,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,Centro-Oeste,DF,Brasília,2020-08-20,3015268,143759,Distrito Federal,Brasília,Brasília,5802.0,146,21316,3112136
4073,Centro-Oeste,DF,Brasília,2020-08-21,3015268,145452,Distrito Federal,Brasília,Brasília,5802.0,147,21609,3176523
4074,Centro-Oeste,DF,Brasília,2020-08-22,3015268,147127,Distrito Federal,Brasília,Brasília,5802.0,148,21904,3241792
4075,Centro-Oeste,DF,Brasília,2020-08-23,3015268,148998,Distrito Federal,Brasília,Brasília,5802.0,149,22201,3307949


In [9]:
#pop_dense = population as of 2019 / SQ_KM
#convert population to int because it is saved as a string
COVID_data_caps['pop_dense'] = COVID_data_caps['Population as of 2019'].astype(int)/COVID_data_caps['SQ_KM']
COVID_data_caps['pop_dense_sq'] = COVID_data_caps['pop_dense'] * COVID_data_caps['pop_dense']


COVID_data_caps

,Region,State,Municipality,Date,Population as of 2019,Accumulated cases,State_Name,Capitol City,City,SQ_KM,days,days_sq,days_cube,pop_dense,pop_dense_sq
0,Norte,RO,Porto Velho,2020-03-27,529544,0,Rondônia,Porto Velho,Porto Velho,34091.0,0,0,0,15.533249,241.281832
1,Norte,RO,Porto Velho,2020-03-28,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,1,1,1,15.533249,241.281832
2,Norte,RO,Porto Velho,2020-03-29,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,2,4,8,15.533249,241.281832
3,Norte,RO,Porto Velho,2020-03-30,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,3,9,27,15.533249,241.281832
4,Norte,RO,Porto Velho,2020-03-31,529544,6,Rondônia,Porto Velho,Porto Velho,34091.0,4,16,64,15.533249,241.281832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,Centro-Oeste,DF,Brasília,2020-08-20,3015268,143759,Distrito Federal,Brasília,Brasília,5802.0,146,21316,3112136,519.694588,270082.464872
4073,Centro-Oeste,DF,Brasília,2020-08-21,3015268,145452,Distrito Federal,Brasília,Brasília,5802.0,147,21609,3176523,519.694588,270082.464872
4074,Centro-Oeste,DF,Brasília,2020-08-22,3015268,147127,Distrito Federal,Brasília,Brasília,5802.0,148,21904,3241792,519.694588,270082.464872
4075,Centro-Oeste,DF,Brasília,2020-08-23,3015268,148998,Distrito Federal,Brasília,Brasília,5802.0,149,22201,3307949,519.694588,270082.464872


In [10]:
#select only capitol cities
temp_by_capitol = temp_by_state[temp_by_state['IS_CAPITOL'].str.lower()=='y']

#select only annual temp
temp_by_capitol = temp_by_capitol[['STATE_ABBR','ANNUAL']]
temp_by_capitol

,STATE_ABBR,ANNUAL
5,AC,76.6
6,AL,76.6
8,AP,79.9
10,AM,81.0
12,BA,77.5
14,CE,79.9
16,DF,69.1
18,ES,75.6
20,GO,73.8
22,MA,79.0


In [11]:
#merge the temperature data onto the COVID data
COVID_data_caps=COVID_data_caps.merge(temp_by_capitol, left_on='State', right_on='STATE_ABBR')
COVID_data_caps

,Region,State,Municipality,Date,Population as of 2019,Accumulated cases,State_Name,Capitol City,City,SQ_KM,days,days_sq,days_cube,pop_dense,pop_dense_sq,STATE_ABBR,ANNUAL
0,Norte,RO,Porto Velho,2020-03-27,529544,0,Rondônia,Porto Velho,Porto Velho,34091.0,0,0,0,15.533249,241.281832,RO,78.1
1,Norte,RO,Porto Velho,2020-03-28,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,1,1,1,15.533249,241.281832,RO,78.1
2,Norte,RO,Porto Velho,2020-03-29,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,2,4,8,15.533249,241.281832,RO,78.1
3,Norte,RO,Porto Velho,2020-03-30,529544,5,Rondônia,Porto Velho,Porto Velho,34091.0,3,9,27,15.533249,241.281832,RO,78.1
4,Norte,RO,Porto Velho,2020-03-31,529544,6,Rondônia,Porto Velho,Porto Velho,34091.0,4,16,64,15.533249,241.281832,RO,78.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,Centro-Oeste,DF,Brasília,2020-08-20,3015268,143759,Distrito Federal,Brasília,Brasília,5802.0,146,21316,3112136,519.694588,270082.464872,DF,69.1
4073,Centro-Oeste,DF,Brasília,2020-08-21,3015268,145452,Distrito Federal,Brasília,Brasília,5802.0,147,21609,3176523,519.694588,270082.464872,DF,69.1
4074,Centro-Oeste,DF,Brasília,2020-08-22,3015268,147127,Distrito Federal,Brasília,Brasília,5802.0,148,21904,3241792,519.694588,270082.464872,DF,69.1
4075,Centro-Oeste,DF,Brasília,2020-08-23,3015268,148998,Distrito Federal,Brasília,Brasília,5802.0,149,22201,3307949,519.694588,270082.464872,DF,69.1


In [12]:
# Get Final Features DataFrame
#select only relevant columns and rename as needed to meet requirements
features = COVID_data_caps[['days_cube','days_sq','days','ANNUAL','pop_dense_sq','pop_dense','Population as of 2019']]
features = features.rename({'ANNUAL':'temp','Population as of 2019':'pop'}, axis=1)
features


,days_cube,days_sq,days,temp,pop_dense_sq,pop_dense,pop
0,0,0,0,78.1,241.281832,15.533249,529544
1,1,1,1,78.1,241.281832,15.533249,529544
2,8,4,2,78.1,241.281832,15.533249,529544
3,27,9,3,78.1,241.281832,15.533249,529544
4,64,16,4,78.1,241.281832,15.533249,529544
...,...,...,...,...,...,...,...
4072,3112136,21316,146,69.1,270082.464872,519.694588,3015268
4073,3176523,21609,147,69.1,270082.464872,519.694588,3015268
4074,3241792,21904,148,69.1,270082.464872,519.694588,3015268
4075,3307949,22201,149,69.1,270082.464872,519.694588,3015268


In [13]:
# Get Final Response DataFrame
#make sure to use double brackets to get a DataFrame instead of a Series
response = COVID_data_caps[['Accumulated cases']]
response

,Accumulated cases
0,0
1,5
2,5
3,5
4,6
...,...
4072,143759
4073,145452
4074,147127
4075,148998


# Prepare DataFrames for Grading

**Do not make changes to the below code**

After completing all data preparation tasks, run the following four cells to prepare your DataFrame for grading by:

1. Outputting the `features` and `response` DataFrames (you do not need to print).

2. Using the NumPy [around()](https://numpy.org/doc/stable/reference/generated/numpy.around.html) function to round all the values in both DataFrames to ___ZERO decimal places___.  You are calling these `features_round` and `response_round`, respectively.

3. Computing the sum of every column for both `features_round` and `response_round`, and saving those values as `features_final` and `response_final`.

___Finally, you are printing your final answer using the `print()` function.___

___Be sure to run all cells of your notebook prior to submitting, so that all output is rendered, visible and there are no error messages.___


In [14]:
features

,days_cube,days_sq,days,temp,pop_dense_sq,pop_dense,pop
0,0,0,0,78.1,241.281832,15.533249,529544
1,1,1,1,78.1,241.281832,15.533249,529544
2,8,4,2,78.1,241.281832,15.533249,529544
3,27,9,3,78.1,241.281832,15.533249,529544
4,64,16,4,78.1,241.281832,15.533249,529544
...,...,...,...,...,...,...,...
4072,3112136,21316,146,69.1,270082.464872,519.694588,3015268
4073,3176523,21609,147,69.1,270082.464872,519.694588,3015268
4074,3241792,21904,148,69.1,270082.464872,519.694588,3015268
4075,3307949,22201,149,69.1,270082.464872,519.694588,3015268


In [15]:
response

,Accumulated cases
0,0
1,5
2,5
3,5
4,6
...,...
4072,143759
4073,145452
4074,147127
4075,148998


In [16]:
features_round = np.around(features, decimals=0)
features_final = features_round.sum(axis=0)
print(features_final)

days_cube                                              3462901875
days_sq                                                  30679425
days                                                       305775
temp                                                     309550.0
pop_dense_sq                                        61867551182.0
pop_dense                                              11277284.0
pop             5295445295445295445295445295445295445295445295...
dtype: object


In [17]:
response_round = np.around(response, decimals=0)
response_final = response_round.sum(axis=0)
print(response_final)

Accumulated cases    61281356
dtype: int64
